In [1]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertForSequenceClassification, BertConfig, get_linear_schedule_with_warmup
from datasets import load_dataset
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import shutil
import unicodedata
from WordPieceTokenizer import WordPieceTokenizer

MAX_SEQUENCE_LENGTH = 128
FINE_TUNE_BATCH_SIZE = 16 # 파인튜닝 시 배치 크기
FINE_TUNE_EPOCHS = 3 # 파인튜닝 에폭
FINE_TUNE_LR = 5e-5 # 파인튜닝 학습률

MODEL_PATH = "./Pretrained"
VOCAB_FILE_PATH = os.path.join(MODEL_PATH, 'vocab.txt')

In [2]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_dataset):
        self.input_ids = torch.tensor(tokenized_dataset["input_ids"], dtype=torch.long)
        self.attention_mask = torch.tensor(tokenized_dataset["attention_mask"], dtype=torch.long)
        self.token_type_ids = torch.tensor(tokenized_dataset["token_type_ids"], dtype=torch.long)
        self.labels = torch.tensor(tokenized_dataset["label"], dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "token_type_ids": self.token_type_ids[idx],
            "labels": self.labels[idx]
        }

In [3]:
# 사용자 정의 토크나이저 로드
print("사용자 정의 WordPieceTokenizer 로드 중...")
try:
    tokenizer = WordPieceTokenizer(vocab_file_path=VOCAB_FILE_PATH, do_lower_case=False, strip_accents=False, clean_text=True)
    print(f"사용자 정의 WordPieceTokenizer 로드 완료. 어휘집 크기: {tokenizer.get_vocab_size()}")
except Exception as e:
    print(f"오류: 토크나이저 로드 중 오류 발생: {e}")
    print(f"오류: '{VOCAB_FILE_PATH}' 파일이 '{MODEL_PATH}'에 올바르게 존재하는지 확인하세요.")
    exit()
    
# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"모델을 '{device}' 장치로 로드하고 파인튜닝을 시작합니다.")

# --- 4. IMDb 데이터셋 로드 및 전처리 ---
print("IMDb 데이터셋 로드 중...")
raw_imdb_dataset = load_dataset("imdb")
print("IMDb 데이터셋 로드 완료.")

def tokenize_function(examples):
    encoded_batch = {
        "input_ids": [],
        "attention_mask": [],
        "token_type_ids": []
    }

    for text_item in examples["text"]:
        if not isinstance(text_item, str):
            print(f"경고: 예상치 못한 데이터 타입 발견: {type(text_item)}. 빈 문자열로 대체합니다.")
            text_item = ""

        encoded_input = tokenizer.encode(
            text_item,
            max_length=MAX_SEQUENCE_LENGTH,
            truncation=True,
            padding="max_length"
        )
        encoded_batch["input_ids"].append(encoded_input["input_ids"])
        encoded_batch["attention_mask"].append(encoded_input["attention_mask"])
        encoded_batch["token_type_ids"].append(encoded_input["token_type_ids"])
    
    return encoded_batch

print("IMDb 데이터셋 토큰화 중...")
tokenized_imdb_dataset = raw_imdb_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=1,
    remove_columns=["text"]
)
print("IMDb 데이터셋 토큰화 완료.")


사용자 정의 WordPieceTokenizer 로드 중...
사용자 정의 WordPieceTokenizer 로드 완료. 어휘집 크기: 32000
모델을 'cuda' 장치로 로드하고 파인튜닝을 시작합니다.
IMDb 데이터셋 로드 중...
IMDb 데이터셋 로드 완료.
IMDb 데이터셋 토큰화 중...
IMDb 데이터셋 토큰화 완료.


In [4]:
train_imdb_dataset = IMDbDataset(tokenized_imdb_dataset["train"])
eval_imdb_dataset = IMDbDataset(tokenized_imdb_dataset["test"])

train_dataloader = DataLoader(train_imdb_dataset, shuffle=True, batch_size=FINE_TUNE_BATCH_SIZE)
eval_dataloader = DataLoader(eval_imdb_dataset, shuffle=False, batch_size=FINE_TUNE_BATCH_SIZE)

print(f"IMDb 학습 데이터로더 준비 완료 (배치 크기: {FINE_TUNE_BATCH_SIZE})")
print(f"IMDb 평가 데이터로더 준비 완료 (배치 크기: {FINE_TUNE_BATCH_SIZE})")



IMDb 학습 데이터로더 준비 완료 (배치 크기: 16)
IMDb 평가 데이터로더 준비 완료 (배치 크기: 16)


In [5]:
# --- 5. 모델 로드 및 파인튜닝 준비 ---
print(f"'{MODEL_PATH}'에서 사전 학습된 모델 로드 중...")
# BertForSequenceClassification은 Classification 헤드가 추가됩니다.
# num_labels는 IMDb 데이터셋의 클래스 수 (긍정/부정 = 2)
model = BertForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=2)
model.to(device)
print("모델 로드 완료.")

optimizer = AdamW(model.parameters(), lr=FINE_TUNE_LR)
total_steps = len(train_dataloader) * FINE_TUNE_EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./Pretrained and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'./Pretrained'에서 사전 학습된 모델 로드 중...
모델 로드 완료.


In [6]:
# --- 6. 모델 파인튜닝 ---
print(f"\n--- 모델 파인튜닝 시작 ({FINE_TUNE_EPOCHS} 에폭) ---")
model.train()
for epoch in range(FINE_TUNE_EPOCHS):
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Fine-tune Epoch {epoch+1}")
    for step, batch in enumerate(progress_bar):
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batc h)
        loss = outputs.loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Fine-tune Epoch {epoch+1} 완료. 평균 학습 손실: {avg_train_loss:.4f}")

print("\n--- 파인튜닝 완료. 모델 평가 시작 ---")




--- 모델 파인튜닝 시작 (3 에폭) ---


Fine-tune Epoch 1: 100%|██████████████████████████████████████████████| 1563/1563 [07:02<00:00,  3.70it/s, loss=0.6349]


Fine-tune Epoch 1 완료. 평균 학습 손실: 0.5948


Fine-tune Epoch 2: 100%|██████████████████████████████████████████████| 1563/1563 [09:01<00:00,  2.89it/s, loss=0.3630]


Fine-tune Epoch 2 완료. 평균 학습 손실: 0.5078


Fine-tune Epoch 3: 100%|██████████████████████████████████████████████| 1563/1563 [05:12<00:00,  5.00it/s, loss=0.5637]

Fine-tune Epoch 3 완료. 평균 학습 손실: 0.4368

--- 파인튜닝 완료. 모델 평가 시작 ---


In [8]:
# --- 7. 모델 평가 ---
model.eval()
predictions = []
true_labels = []

eval_progress_bar = tqdm(eval_dataloader, desc="Evaluating")
for batch in eval_progress_bar:
    batch = {k: v.to(device) for k, v in batch.items()}
    
    with torch.no_grad():
        outputs = model(**batch)
    
    logits = outputs.logits
    preds = torch.argmax(logits, dim=-1)

    predictions.extend(preds.cpu().numpy())
    true_labels.extend(batch["labels"].cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='binary') # IMDb는 이진 분류

print(f"\n===== 모델 평가 결과 =====")
print(f"정확도 (Accuracy): {accuracy:.4f}")
print(f"F1 점수 (F1-Score): {f1:.4f}")
print(f"=========================")

Evaluating: 100%|██████████████████████████████████████████████████████████████████| 1563/1563 [01:25<00:00, 18.33it/s]


===== 모델 평가 결과 =====
정확도 (Accuracy): 0.7704
F1 점수 (F1-Score): 0.7763

[참고]: 이 성능은 여러분의 사전 학습 모델의 품질, 파인튜닝 데이터셋의 크기, 파인튜닝 하이퍼파라미터에 따라 크게 달라집니다.
베이스라인 성능과 비교하기 위해 Hugging Face의 `bert-base-uncased` 모델을 IMDb에 파인튜닝했을 때의 성능과 비교해 볼 수 있습니다.


In [9]:
torch.save(model.state_dict(),"saves/Pretrain.pt")